## Ejercicios de análisis exploratorio de datos con Python Pandas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark
import math

### Ejercicio 1

** 2017 1er cuatrimestre (primera oportunidad)**

Un sitio de Ebooks tiene información sobre los reviews que los usuarios hacen de sus libros en un DataFrame con formato (user_id, book_id, rating, timestamp). Por otro lado tenemos información en otro DataFrame que bajamos de GoodReads: (book_id, book_name, avg_rating). Podemos suponer que los Ids de los libros son compatibles. Se pide usar Python Pandas para:

  a) obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks.
(Para este punto se puede ignorar el segundo DataFrame) (7,5 ptos);

  b) obtener un DataFrame que indique qué libros tienen una diferencia de rating
promedio mayor al 20% entre el sitio de Ebooks y GoodReads (7,5 ptos).

#### Criterio de corrección

Para el punto a) Si no tiene en cuenta un mínimo de ratings por libro antes de considerar el promedio -3. Si
agrupa mal o calcula mal el promedio -5, si la lógica está mal vale cero. Errores mínimos de sintaxis no serán tenidos
en cuenta, pero si utilizan funciones no existentes se descuentan puntos en función de que hagan. Para el punto b)
Al igual que antes, si calculan el promedio sin evaluar la cantidad de ratings para un book_id dado -3, si hace mal el
join/merge -5, si hacen mal la validación de la condición -3.


#### Lectura de los datos

In [2]:
ebooks = pd.read_csv('./data/ebooks/ebooks.csv', encoding = 'utf-8')
ebooks.head()

,user_id,book_id,rating,timestamp
0,4,2,9,2017-05-04
1,2,1,3,2017-05-03
2,2,2,5,2017-09-21
3,3,1,6,2018-01-15
4,5,3,2,2016-12-24


In [3]:
goodreads = pd.read_csv('./data/ebooks/goodreads.csv', encoding = 'utf-8')
goodreads.head()

,book_id,book_name,avg_rating
0,1,20000 leguas de viaje submarino,8.4
1,2,Estudio en escarlata,2.3
2,3,La isla misteriosa,9.2
3,4,La isla del tesoro,7.6
4,5,Las aventuras de Tom Sawyer,6.7


#### a) Top 5 del primer data frame

In [4]:
# Me quedo con las columnas que me interesan
dfa = ebooks.loc[:,['book_id', 'rating']]

# Filtro los que tienen un solo review
dfa = dfa.groupby('book_id').filter(lambda x: len(x) > 1)

# Agrupo por book_id, calculo el promedio, ordeno
# descendientemente y tomo los primeros cinco
dfa.groupby('book_id').mean()\
    .sort_values('rating', ascending = False).head(5)

,rating
book_id,
4,7.0
2,6.5
5,6.5
7,5.5
1,5.0


##### Otra forma

In [5]:
# Me quedo con las columnas que me interesan
dfa = ebooks.loc[:,['book_id', 'rating']]

# Agrupo por libro, cuento los votos y obtengo una
# serie con la cantidad de votos por libro
num_reviews = dfa.groupby('book_id').count()['rating']

# Agrupo por libro y calculo el promedio del puntaje
dfa = dfa.groupby('book_id').mean()

# Agrego la columna de cantidad de reviews
dfa['reviews'] = num_reviews

# Descarto los libros que tengan un solo review
dfa = dfa.loc[dfa['reviews'] > 1,:]

# Ordeno descendentemente y muestro los primeros 5
dfa.sort_values(by = 'rating', ascending = False).head(5)

,rating,reviews
book_id,,
4,7.0,3
2,6.5,6
5,6.5,2
7,5.5,2
1,5.0,5


#### b) Libros con una diferencia de rating promedio mayor al 20% entre ambos sitios

In [6]:
# Hago un inner join entre ambos data frames sobre el book_id
merged = pd.merge(dfa.reset_index(), goodreads, on = 'book_id', how = 'inner')
merged

,book_id,rating,reviews,book_name,avg_rating
0,1,5.000000,5,20000 leguas de viaje submarino,8.4
1,2,6.500000,6,Estudio en escarlata,2.3
2,3,2.666667,6,La isla misteriosa,9.2
3,4,7.000000,3,La isla del tesoro,7.6
4,5,6.500000,2,Las aventuras de Tom Sawyer,6.7
5,7,5.500000,2,De la Tierra a la Luna,4.3


In [7]:
# Me quedo solo con aquellos resultados que tengan una diferencia
# mayor a 20% entre ambas columnas de rating
merged.loc[(merged['avg_rating'] < merged['rating'] * 0.8)\
          | (merged['avg_rating'] > merged['rating'] * 1.2),:]

,book_id,rating,reviews,book_name,avg_rating
0,1,5.000000,5,20000 leguas de viaje submarino,8.4
1,2,6.500000,6,Estudio en escarlata,2.3
2,3,2.666667,6,La isla misteriosa,9.2
5,7,5.500000,2,De la Tierra a la Luna,4.3


### Ejercicio 2

** 2017 2do cuatrimestre (primera oportunidad) **

Tenemos un dataframe con la información de distintas playlists armadas por usuarios con el formato (playlist, song_id, description). A su vez, contamos con un dataframe de canciones
que contiene (song_id, singer, year, length, genres).
Se pide generar un programa en Pandas que indique para cada playlist cuál es el cantante predominante (con mas canciones incluidas dentro de esa lista). (15 pts) 

#### Criterio de corrección

Si en algún lado hacen un groupByKey se hace un descuento de 10 puntos. En ningún caso son necesarios los datos de cada registro.
Si filtran después de hacer el resto de las operaciones, descuento de 2 puntos (es mucho más eficiente filtrar antes y solo trabajar con un
conjunto acotado, podríamos tener todos los patentamientos de la historia). 

#### Lectura de los datos

In [8]:
playlists = pd.read_csv('./data/playlists/playlists.csv', encoding='utf-8')
playlists.head()

,playlist,song_id,description
0,1,3,werwe
1,1,5,ijias
2,4,4,oiurewq
3,1,4,fdsa
4,2,4,rewqew


In [9]:
songs = pd.read_csv('./data/playlists/songs.csv', encoding='utf-8')
songs.head()

,song_id,singer,year,length,genres
0,1,Michael Jackson,1985,8,Pop
1,2,Elton John,1984,2,Pop
2,3,Ronnie Dio,1987,3,Metal
3,4,Elton John,1982,4,Pop
4,5,Elton John,1985,3,Pop


#### Resolución agrupando y buscando el máximo

In [10]:
# Me deshago primero de las columnas que no me interesan
playlists = playlists.loc[:,['playlist', 'song_id']]
songs = songs.loc[:, ['song_id', 'singer']]

# Hago un inner join sobre song_id
merged = pd.merge(playlists, songs, how = 'inner', on = 'song_id')
merged.head()

,playlist,song_id,singer
0,1,3,Ronnie Dio
1,2,3,Ronnie Dio
2,3,3,Ronnie Dio
3,1,5,Elton John
4,2,5,Elton John


In [11]:
# Hago un groupby por playlist y singer y luego me quedo con el máximo de cada uno
grouped = merged.groupby(['playlist', 'singer'])\
    .size().to_frame('cant').reset_index()
grouped

,playlist,singer,cant
0,1,Elton John,2
1,1,Ronnie Dio,1
2,2,Elton John,3
3,2,Ronnie Dio,1
4,3,Elton John,1
5,3,Ronnie Dio,1
6,4,Elton John,2
7,5,Michael Jackson,2


In [12]:
# Me quedo con el máximo por cada playlist
grouped.groupby('playlist')\
    .apply(lambda x: x[x['cant'] == x['cant'].max()])

playlist           singer  cant
playlist                                   
1        0         1       Elton John     2
2        2         2       Elton John     3
3        4         3       Elton John     1
         5         3       Ronnie Dio     1
4        6         4       Elton John     2
5        7         5  Michael Jackson     2

#### Resolución agrupando, ordenando y eliminando duplicados

In [13]:
# Me deshago primero de las columnas que no me interesan
playlists = playlists.loc[:,['playlist', 'song_id']]
songs = songs.loc[:, ['song_id', 'singer']]

# Hago un inner join sobre song_id
merged = pd.merge(playlists, songs, how = 'inner', on = 'song_id')

# Agrupo por playlist y singer, los cuento, ordeno
# descendientemente y elimino los duplicados
df = merged.groupby(['playlist','singer']).size()\
    .to_frame('cantidad').reset_index()\
    .sort_values('cantidad', ascending = False)\
    .drop_duplicates('playlist')

df.set_index('playlist')

,singer,cantidad
playlist,,
2,Elton John,3
1,Elton John,2
4,Elton John,2
5,Michael Jackson,2
3,Elton John,1


#### Resolución con Spark

In [ ]:
# (playlist, song_id, desc) (0,1,2,3)
# (song_id, singer, year, length, genres) (0,1,2,3,4)

playlists = playlists.map(lambda x: (x[1],x[0]))
songs = songs.map(lambda x: (x[0],x[1]))
merge = songs.join(playlists)
merge = merge.map(lambda x: ((x[1][1],x[1][0]),1))
merge = merge.reduceByKey(lambda a,b: a + b)

merge = merge.map(lambda x: (x[0][0],(x[0][1],x[1])))
merge = merge.reduce(lambda a,b: if a[1]>b[1] a else b)

### Ejercicio 3

** 2017 2do cuatrimestre (segunda oportunidad) **

La Agencia Nacional de Estadísticas de Buenos Aires recolecta información de nacimientos cuando los padres registran a sus hijos en el registro civil a partir de una encuesta. Esa información se encuentra disponible para su análisis en un csv con el siguiente formato
(dia_nacimiento, mes_nacimiento, anio_nacimiento, peso_al_nacer, longitud_al_nacer, id_hospital, tipo_parto), donde el tipo de parto 1 es natural y 2 es cesárea.
Por otro lado la agencia cuenta con información histórica de los hospitales en otro csv con siguiente formato (id_hospital, dirección, promedio_nacimientos_mensual).
Se pide usar Pandas para:

  a) Calcular la cantidad de nacimientos para cada uno de los hospitales para el mes de Octubre de 2017 e indicar aquellos hospitales que superan el promedio de nacimientos mensuales.
  
  b) Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el % de cesáreas con respecto a ese mes del año 2016.
  
#### Criterio de corrección

El enunciado dice que se cuenta con dos csv, por lo que hay que comenzar leyéndolos. a) Por un lado es necesario filtrar del Data frame
aquellos datos que corresponden al mes de Octubre de 2017. Para obtener la cantidad de nacimientos, por hospital se puede hacer de muchas
formas, una es agrupar y usar size, otra opción seria generar una columna con 1 por cada nacimiento y realizar una agrupación por hospital
realizando un aggregate(sum) de esa columna para tener el total por hospital.
Luego es necesario realizar el un inner join con la información histórica de hospitales y filtrar aquellas que no cuyo total sea menor o igual al
promedio histórico de nacimientos.
b) Se debe trabajar respectivamente filtrando datos del data frame para los meses de octubre de los años 2016 y 2017. Es necesario obtener el
total de nacimientos para cada uno de los meses y el total de cesáreas de ambos casos para poder calcular el porcentaje para cada año.
Es importante que se determine sí se incrementó o no el porcentaje. Si no se realiza descuento de 4 puntos.

#### Lectura de datos

In [14]:
nacimientos = pd.read_csv('./data/nacimientos/nacimientos.csv')
nacimientos.head()

,dia_nacimiento,mes_nacimiento,anio_nacimiento,peso_al_nacer,longitud_al_nacer,id_hospital,tipo_parto
0,1,10,2016,3,0.5,4,1
1,2,10,2017,3,0.5,4,2
2,3,9,2005,3,0.5,4,1
3,4,8,2017,3,0.5,4,1
4,5,10,2016,3,0.5,4,1


In [15]:
historico = pd.read_csv('./data/nacimientos/historico_mensual.csv')
historico.head()

,id_hospital,dirección,promedio_nacimientos_mensual
0,4,Rivadavia 1234,5
1,1,9 de Julio 1000,4
2,3,Evergreen 123,8
3,2,General Paz 984,10


#### a) Nacimientos en octubre de 2017 que superan el histórico mensual

In [16]:
# Me quedo con los nacimientos en octubre de 2017 
# y con las columnas id_hospital y tipo_parto
nacs = nacimientos.loc[(nacimientos.anio_nacimiento == 2017) & \
                        (nacimientos.mes_nacimiento == 10),\
                        ['id_hospital', 'tipo_parto']]
nacs.head()

,id_hospital,tipo_parto
1,4,2
6,4,1
8,4,1
15,4,1
17,4,1


In [17]:
# Agrupo por hospitales y calculo la cantidad
nacs = nacs.groupby('id_hospital').size().to_frame('cant').reset_index()
nacs

,id_hospital,cant
0,1,2
1,3,1
2,4,7


In [18]:
# Hago un inner join con el dataframe histórico
merged = pd.merge(nacs, historico, on = 'id_hospital', how = 'inner')

# Me quedo con los hospitales que superen el histórico mensual
merged.loc[merged.cant > merged.promedio_nacimientos_mensual,:]

,id_hospital,cant,dirección,promedio_nacimientos_mensual
2,4,7,Rivadavia 1234,5


#### b) Incremento de cesáreas entre octubre de 2016 y 2017

In [19]:
nacs10 = nacimientos.loc[((nacimientos.anio_nacimiento == 2016) |
                (nacimientos.anio_nacimiento == 2017)) & \
                (nacimientos.mes_nacimiento == 10), \
                ['anio_nacimiento','tipo_parto']]
nacs10.head()

,anio_nacimiento,tipo_parto
0,2016,1
1,2017,2
4,2016,1
6,2017,1
7,2016,1


In [21]:
nacs16 = nacimientos.loc[(nacimientos.anio_nacimiento == 2016) & \
                  (nacimientos.mes_nacimiento == 10), ['tipo_parto']]
nacs16.head()

,tipo_parto
0,1
4,1
7,1
10,2
13,1


In [22]:
nacs16 = nacs16.groupby('tipo_parto').size().to_frame('cant')
nacs16

,cant
tipo_parto,
1,5
2,2


In [23]:
nacs16['perc'] = nacs16.apply(lambda x: x.cant / nacs16.cant.agg(sum), axis = 1)
nacs16

,cant,perc
tipo_parto,,
1,5,0.714286
2,2,0.285714


In [25]:
nacs17 = nacimientos.loc[(nacimientos.anio_nacimiento == 2017) & \
                  (nacimientos.mes_nacimiento == 10), ['tipo_parto']]
nacs17 = nacs17.groupby('tipo_parto').size().to_frame('cant')
nacs17['perc'] = nacs17.apply(lambda x: x.cant / nacs17.cant.agg(sum), axis = 1)
nacs17

,cant,perc
tipo_parto,,
1,5,0.5
2,5,0.5


In [26]:
merged = pd.merge(nacs16.reset_index(), nacs17.reset_index(),\
                  on = 'tipo_parto', how = 'inner')
merged

,tipo_parto,cant_x,perc_x,cant_y,perc_y
0,1,5,0.714286,5,0.5
1,2,2,0.285714,5,0.5
